<a href="https://colab.research.google.com/github/Valdini/COURSERA_CAPSTONE/blob/main/Capstone_Project_Toronto_Bar_Supplier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Introduction / Business Problem**

The globally operating bar equipment supplier "Bar10der" is entering the Canadian market and is targeting large Canadian cities to supply local bars with bar equipment (e.g. chairs, beer taps). Bar10der is a young, asset-light business reducing operational cost by delivering bar supply without a truck or car fleet. Bar10der reduces the distance between warehouse and bar by renting a multi-purpose location serving as office space and warehouse in the city center. In order to determine the optimal warehouse location, Bar10der looks for the city area that contains the highest bar density.


**Data & Methods used**

In order to solve the problem, data from 3 data sources will be used: location data of bars in Toronto fetched from the Foursquare API, neighborhood identification data from Wikipedia and additional geospatial data. By identifying those Toronto postal codes that have the highest bar density (determined by the number of bars), the ideal Toronto area (as postal code) will be identified and visualized on a Folium map based on geolocation data.

**Creating a Wikipedia-based list of boroughs in Toronto**

In [182]:
#Creating a Wikipedia-based list of boroughs and neighborhoods in Toronto

import pandas as pd #importing the pandas library
import numpy as np #importing the numpy library
df_scrape = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') #telling pandas to find URL
df_table = df_scrape[0] #telling pandas to take table
df_toronto = df_table[['Postal Code','Borough','Neighbourhood']] #taking table columns
df_toronto.columns = ['Postal Code','Borough','Neighborhood'] #setting/renaming table columns
df_toronto_cleaned = df_toronto[df_toronto.Borough != 'Not assigned'] #ignoring not assigned values for borough
df_toronto_cleaned.groupby('Postal Code') #showing all neighborhoods per postal code
df_tc = df_toronto_cleaned #shorting the table name
df_tc['Neighborhood'] = np.where(df_tc['Neighborhood'] == 'Not assigned', df_tc['Borough'], df_tc['Neighborhood']) #replacing not assigned values of neighborhood with their borough values
df_tc.index = df_tc.index - 2 #cleaning index
df_tc

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
158,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
163,M4Y,Downtown Toronto,Church and Wellesley
166,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
167,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


**Merging Wikipedia-based list with geospatial data**

In [183]:
#Loading geo data and merging it to Wikipedia table

df_geo = pd.read_csv('http://cocl.us/Geospatial_data') #creating a new dataframe using the geospatial data csv file
df_wp = df_tc.merge(df_geo,on='Postal Code',how = 'left') #adding the latitude and longitue by merging on the common postal code column
df_wp

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


**Creating a Foursquare-based list of bars in Toronto**

In [184]:
#Performing necessary imports

import requests # library to handle requests
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [185]:
#Passing Foursquare API access data

CLIENT_ID = 'QUUMEYFVW03TKZNFU4OD15I1EC45ZZ0VI1P5ZMENZBIFGJA1' #Foursquare ID
CLIENT_SECRET = 'U1TGTC0RS0PQ4GVUD5LZDS5DMZDWQHI1KUGXSANWEM014INU' #Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [186]:
#Building the Foursquare API search query in Toronto and initializing Folium map

address = 'Toronto'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
location_map = folium.Map(location=[latitude, longitude], zoom_start=15)

In [187]:
#Building the Foursquare API search query for bars

search_query = 'Bar'
radius = 10000
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()

Bar .... OK!


In [188]:
# assign relevant part of JSON to venues

venues = results['response']['venues']

# tranform venues into a dataframe

df_fs = json_normalize(venues)
df_fs['location.postalCode'] = df_fs['location.postalCode'].str[:3]
df_fs.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,venuePage.id
0,4af4e6e2f964a52052f721e3,St. Louis Bar & Grill,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",v-1603830400,False,595 Bay St #A09,Atrium on Bay,43.656562,-79.382737,"[{'label': 'display', 'lat': 43.65656209045694...",356,M5G,CA,Toronto,ON,Canada,"[595 Bay St #A09 (Atrium on Bay), Toronto ON M...",NaN,NaN
1,5612ea8b498e102e764fc304,Bar Hop Brewco,"[{'id': '56aa371ce4b08b9a8d57356c', 'name': 'B...",v-1603830400,False,137 Peter St,Peter & Richmond,43.649069,-79.393456,"[{'label': 'display', 'lat': 43.64906884285040...",910,M5V,CA,Toronto,ON,Canada,"[137 Peter St (Peter & Richmond), Toronto ON M...",NaN,NaN
2,52f6816f11d24a43115dc834,Scaddabush Italian Kitchen & Bar,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1603830400,False,"382 Yonge Street, Unit #7",Gerrard,43.658920,-79.382891,"[{'label': 'display', 'lat': 43.65892029202872...",611,M5B,CA,Toronto,ON,Canada,"[382 Yonge Street, Unit #7 (Gerrard), Toronto ...",NaN,NaN
3,4d2b615e342d6dcb2b8115cb,Earls Kitchen & Bar,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",v-1603830400,False,150 King St. W.,at York St.,43.647946,-79.383706,"[{'label': 'display', 'lat': 43.64794566298030...",616,M5H,CA,Toronto,ON,Canada,"[150 King St. W. (at York St.), Toronto ON M5H...",NaN,NaN
4,57dd99cb498ee67580d16390,Bar Verde,"[{'id': '4bf58dd8d48988d157941735', 'name': 'N...",v-1603830400,False,260 Yonge Street,NaN,43.654837,-79.380742,"[{'label': 'display', 'lat': 43.654837, 'lng':...",298,M5B,CA,Toronto,ON,Canada,"[260 Yonge Street, Toronto ON M5B 2L9, Canada]",NaN,NaN


**Merging Wikipedia/Geodata and Foursquare tables on postal code**

In [193]:
#Creating the merged table and grouping it by postal code occurrence to see highest bar density postal codes

df_comb = pd.merge(df_fs,df_wp,left_on = df_fs['location.postalCode'], right_on = df_wp['Postal Code'], how = 'left')

**Results and discussion**

In [194]:
df_comb.groupby(df_comb['Postal Code']).size().sort_values(ascending=[False])

Postal Code
M5H    6
M5B    5
M5V    4
M5J    2
M5E    2
M5C    2
M5T    1
M5G    1
M4Y    1
dtype: int64

The analysis shows that postal code M5H has the highest bar density with 6 bars. In the following, the result is visualized.

**Visualizing the result on a Folium map**

In [190]:
#Overview of postal codes

postcodes = df_comb['Postal Code'].unique().tolist()
postcodes

['M5G', 'M5V', 'M5B', 'M5H', 'M5J', nan, 'M5C', 'M5T', 'M5E', 'M4Y']

In [191]:
#Mapping postal codes to fill colours for visualization

color_dict = {'M5G': 'lightgreen', 
'M5B': 'red', 
'M5H': 'blue', 
'M5J': 'purple', 
'M5C': 'orange',
'nan': 'darkred',
'M5E': 'lightred',
'M5T': 'beige',
'M4Y': 'darkblue'}
df_comb['fillcolor'] = df_comb['Postal Code'].map(color_dict)

In [192]:
#Creating the Folium map with popup showing the postal code

df_comb.dropna(subset = ["fillcolor"], inplace=True)
Latitudelist = df_comb['location.lat'].astype(float).values.tolist()
Longitudelist = df_comb['location.lng'].astype(float).values.tolist()
Colorlist = df_comb['fillcolor'].values.tolist()
PClist = df_comb['Postal Code'].values.tolist()
for lat,lng,col,pc in zip(Latitudelist,Longitudelist,Colorlist,PClist):
  label = folium.Popup(str(pc), parse_html=True)
  folium.CircleMarker(
    [lat,lng],
    radius=10,
    fill = True,
    fill_color = col,
    fill_opacity = 0.6,
    popup = label
).add_to(location_map)
location_map